In [3]:
# default_exp pnl_sim

In [1]:
# export

import logging
import pandas as pd

STEP_SIZE = 0.05


def avg_active_signal(signal, binarize, binarize_params):
    if binarize != "fixed_horizon":
        raise ValueError(f"Do not know how to handle binarize method '{binarize}'")
    avg_signal = signal.rolling(binarize_params).mean()
    return discrete_signal(avg_signal, STEP_SIZE)


def get_pnl_reports(events, symbols, binarize, binarize_params):
    closes = []
    clf_signals = []
    alpha_signals = []
    logging.info(
        f"Generating signals for {len(symbols)} symbols on {len(events)} events with b_params={binarize_params}"
    )

    for key, group in events.groupby(events.index.microsecond):
        closes.append(group["close_p"])

        if "side" in group:
            # Meta-labeling
            alpha_signals.append(group["side"])
            signal = group["y_pred"] * group["side"]
        else:
            signal = group["y_pred"]

        clf_s = avg_active_signal(signal, binarize, binarize_params)
        clf_signals.append(clf_s)

    closes = pd.concat(closes, axis=1).ffill()
    closes.columns = symbols

    clf_signals = pd.concat(clf_signals, axis=1).ffill()
    clf_signals.columns = symbols

    if "side" in events:
        alpha_signals = pd.concat(alpha_signals, axis=1).ffill()
        alpha_signals.columns = symbols
    else:
        alpha_signals = None

    return closes, clf_signals, alpha_signals


def discrete_signal(signal0, step_size):
    # discretize signal
    signal1 = (signal0 / step_size).round() * step_size
    return signal1.clip(-1, 1)